In [ ]:
#default_exp callback

In [ ]:
#export
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *
from torchvision.models import vgg16, vgg19
from faststyle import *

In [ ]:
#export
#TODO: hardcoded model
feat_m = vgg19(True).features.cuda().eval()
for p in feat_m.parameters(): p.requires_grad=False

In [ ]:
#export
def VggFeats(layers):
  hooks = hook_outputs(layers, detach=False)
  def _inner(x):
    feat_m(x)
    return hooks.stored
  return _inner

In [ ]:
#export
#TODO: hardcoded layers
# layers = [feat_m[i] for i in [1, 11, 18, 25, 20]]; layers # vgg16
layers = [feat_m[i] for i in [1, 6, 11, 20, 29, 22]]; layers # vgg19
get_fs = VggFeats(layers)

In [ ]:
#export
def _get_ims(fns):
  t = TfmdLists(fns, tfms=[PILImage.create, ToTensor(), IntToFloatTensor(), 
                               Normalize.from_stats(*coco_stats, cuda=False)])
  return [o.to(default_device()) for o in t]

In [ ]:
#export
class StyleFsCallback(Callback):
  def __init__(self, fns, get_fs, tfms=None): 
    self.sims_fs = [get_fs(o) for o in _get_ims(fns)]
    
  def after_pred(self):
    self.learn.yb = (*self.yb, self.sims_fs)

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.ipynb.
Converted 02_loss.ipynb.
Converted 03_models.ipynb.
Converted 04_learner.ipynb.
Converted 05_callback.ipynb.
